<a href="https://colab.research.google.com/github/helloyaz/weatherwise_Yasmine_Kyaw_19473235/blob/main/weatherwise_Yasmine_Kyaw_19473235_starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌦️ WeatherWise – Starter Notebook

Welcome to your **WeatherWise** project notebook! This scaffold is designed to help you build your weather advisor app using Python, visualisations, and AI-enhanced development.

---

📄 **Full Assignment Specification**  
See [`ASSIGNMENT.md`](ASSIGNMENT.md) or check the LMS for full details.

📝 **Quick Refresher**  
A one-page summary is available in [`resources/assignment-summary.md`](resources/assignment-summary.md).

---

🧠 **This Notebook Structure is Optional**  
You’re encouraged to reorganise, rename sections, or remove scaffold cells if you prefer — as long as your final version meets the requirements.

✅ You may delete this note before submission.



## 🧰 Setup and Imports

This section imports commonly used packages and installs any additional tools used in the project.

- You may not need all of these unless you're using specific features (e.g. visualisations, advanced prompting).
- The notebook assumes the following packages are **pre-installed** in the provided environment or installable via pip:
  - `requests`, `matplotlib`, `pyinputplus`
  - `fetch-my-weather` (for accessing weather data easily)
  - `hands-on-ai` (for AI logging, comparisons, or prompting tools)

If you're running this notebook in **Google Colab**, uncomment the following lines to install the required packages.


In [ ]:
!pip install fetch-my-weather
!pip install hands-on-ai


In [ ]:
import os

os.environ['HANDS_ON_AI_SERVER'] = 'http://ollama.serveur.au'
os.environ['HANDS_ON_AI_MODEL'] = 'granite3.2'
os.environ['HANDS_ON_AI_API_KEY'] = input('Enter your API key: ')

Enter your API key: 


## 📦 Setup and Configuration
Import required packages and setup environment.

In [4]:
!pip install fetch-my-weather hands-on-ai pyinputplus


In [5]:
import requests
import matplotlib.pyplot as plt
import pyinputplus as pyip

from fetch_my_weather import get_weather
from hands_on_ai.chat import get_response

# Add any other setup code here

## 🌤️ Weather Data Functions

In [6]:
weather_data = get_weather('London')
print(weather_data)

current_condition=[CurrentCondition(FeelsLikeC='24', FeelsLikeF='76', cloudcover='50', humidity='57', localObsDateTime='2025-05-12 12:48 PM', observation_time='11:48 AM', precipInches='0.0', precipMM='0.9', pressure='1011', pressureInches='30', temp_C='22', temp_F='72', uvIndex='5', visibility='10', visibilityMiles='6', weatherCode='116', weatherDesc=[WeatherDesc(value='Partly cloudy')], weatherIconUrl=[WeatherIconUrl(value='')], winddir16Point='SSE', winddirDegree='148', windspeedKmph='11', windspeedMiles='7')] nearest_area=[NearestArea(areaName=[AreaName(value='London')], country=[Country(value='United Kingdom')], latitude='51.517', longitude='-0.106', population='7421228', region=[Region(value='City of London Greater London')], weatherUrl=[WeatherIconUrl(value='')])] request=[Request(query='Lat 51.51 and Lon -0.13', type='LatLon')] weather=[DailyForecast(astronomy=[Astronomy(moon_illumination='99', moon_phase='Full Moon', moonrise='09:07 PM', moonset='04:36 AM', sunrise='05:13 AM', 

In [14]:
# Define get_weather_data() function here
def get_weather_data(location, forecast_days=5):
    """
    Retrieve weather data for a specified location.

    Args:
        location (str): City or location name
        forecast_days (int): Number of days to forecast (1-5)

    Returns:
        dict: Weather data including current conditions and forecast
    """

def get_weather_data(location, forecast_days=5):
    weather_data = get_weather(location, forecast_days=forecast_days)
    print(weather_data)


In [22]:
def get_weather_data(city="Perth", forecast_days=3):
  print(data)

## 📊 Visualisation Functions

In [8]:
# Define create_temperature_visualisation() and create_precipitation_visualisation() here
def create_temperature_visualisation(weather_data, output_type='display'):
    """
    Create visualisation of temperature data.

    Args:
        weather_data (dict): The processed weather data
        output_type (str): Either 'display' to show in notebook or 'figure' to return the figure

    Returns:
        If output_type is 'figure', returns the matplotlib figure object
        Otherwise, displays the visualisation in the notebook
    """
    pass


In [9]:

def create_precipitation_visualisation(weather_data, output_type='display'):
    """
    Create visualisation of precipitation data.

    Args:
        weather_data (dict): The processed weather data
        output_type (str): Either 'display' to show in notebook or 'figure' to return the figure

    Returns:
        If output_type is 'figure', returns the matplotlib figure object
        Otherwise, displays the visualisation in the notebook
    """
    pass

## 🤖 Natural Language Processing

In [ ]:
# Define parse_weather_question() and generate_weather_response() here
def parse_weather_question(question):
    """
    Parse a natural language weather question.

    Args:
        question (str): User's weather-related question

    Returns:
        dict: Extracted information including location, time period, and weather attribute
    """
    pass

## 🧭 User Interface

In [ ]:
# Define menu functions using pyinputplus or ipywidgets here

## 🧩 Main Application Logic

In [ ]:
# Tie everything together here
def generate_weather_response(parsed_question, weather_data):
    """
    Generate a natural language response to a weather question.

    Args:
        parsed_question (dict): Parsed question data
        weather_data (dict): Weather data

    Returns:
        str: Natural language response
    """
    pass

## 🧪 Testing and Examples

In [ ]:
# Include sample input/output for each function

## 🗂️ AI Prompting Log (Optional)
Add markdown cells here summarising prompts used or link to AI conversations in the `ai-conversations/` folder.